<a href="https://colab.research.google.com/github/Caarlosgg/CLustering_ComercioMinorista/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parte 1: Carga de Datos y Exploración Inicial
En esta fase, importamos las librerías necesarias y cargamos el dataset para ver su estructura.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Configuración de visualización
%matplotlib inline
sns.set(style="whitegrid")

# Cargar el dataset (ajusta la ruta si es necesario)
df = pd.read_csv('OnlineRetail.csv', encoding='unicode_escape')

# Visualizar las primeras filas y la info general
print("Dimensiones del dataset:", df.shape)
display(df.head())
df.info()

Dimensiones del dataset: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


InvoiceNo: Número de factura (identifica la transacción).

StockCode: Código del producto.

Description: Nombre del artículo.

Quantity: Cantidad comprada (ojo: hay valores negativos, que representan devoluciones).

InvoiceDate: Fecha y hora de la compra.

UnitPrice: Precio por unidad.

CustomerID: Identificador único del cliente (esencial para el clustering).

Country: País del cliente.

Puntos críticos detectados:

Valores Nulos: La columna CustomerID tiene muchos valores faltantes. Como vamos a agrupar clientes, no podemos trabajar con registros que no tengan ID.

Tipos de datos: InvoiceDate debe convertirse a formato de fecha.

Inconsistencias: Hay cantidades y precios negativos que debemos limpiar para no sesgar el análisis monetario.

#Parte 2: Limpieza y Preparación de Datos.

En Data Science, la calidad del modelo depende casi al 100% de la calidad de los datos ("Garbage In, Garbage Out"). Aquí vamos a filtrar lo que no nos sirve y crear variables que necesitamos.

Pasos que realizamos en este bloque de código:
Eliminar Nulos en CustomerID: Si no sabemos quién compró, no podemos asignarlo a un clúster. Eliminamos esos registros.

Filtrar Devoluciones y Errores: Eliminamos filas con Quantity <= 0 (devoluciones) y UnitPrice <= 0 (posibles errores del sistema o regalos). Nos interesa el comportamiento de compra.

Formato de Fecha: Convertimos InvoiceDate a un objeto de tiempo real (datetime) para poder hacer cálculos de días (Recency).

Crear variable 'Amount': Multiplicamos Quantity * UnitPrice. Esto nos da el dinero real gastado por transacción.

In [3]:
# --- Parte 2: Limpieza y Transformación ---

# 1. Eliminar filas sin CustomerID (No podemos clusterizar clientes anónimos)
df.dropna(subset=['CustomerID'], inplace=True)

# 2. Eliminar transacciones con cantidades negativas (Devoluciones) o precio 0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# 3. Convertir la fecha a formato datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d-%m-%Y %H:%M')

# 4. Crear la columna 'Amount' (Monto total de la línea de compra)
df['Amount'] = df['Quantity'] * df['UnitPrice']

# Verificación de resultados
print("Dimensiones después de limpieza:", df.shape)
print(df[['CustomerID', 'InvoiceDate', 'Amount']].head())

Dimensiones después de limpieza: (397884, 9)
   CustomerID         InvoiceDate  Amount
0     17850.0 2010-12-01 08:26:00   15.30
1     17850.0 2010-12-01 08:26:00   20.34
2     17850.0 2010-12-01 08:26:00   22.00
3     17850.0 2010-12-01 08:26:00   20.34
4     17850.0 2010-12-01 08:26:00   20.34


#Explicación de la Salida:

Dimensiones: Hemos pasado de ~541,000 filas a 397,884. Hemos perdido datos, sí, pero los que quedan son datos limpios y asignables a clientes reales.

Nueva Columna Amount: Ahora tenemos una cifra monetaria clara por cada línea de producto, lista para sumar por cliente.